In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

In [ ]:
df = pd.read_excel('../metrc/dreamfields outgoing wholesale transfers.xlsx')

In [ ]:
df.info()

In [ ]:
sns.boxplot(x=df['shipper_wholesale_price'])

In [ ]:
df[df['shipper_wholesale_price'] > 1000000]

In [ ]:
df.drop([29469], inplace=True)

In [ ]:
sns.boxplot(x=df['shipper_wholesale_price'])

In [ ]:
df[df['shipper_wholesale_price'] < 1]

In [ ]:
df_1 = df[df['shipper_wholesale_price'] > 1]

In [ ]:
df_1

In [ ]:
sns.boxplot(x=df_1['shipper_wholesale_price'])

In [ ]:
#sum wholesale price off of date

In [ ]:
df_month = df_1.resample('M', on='created_date').shipper_wholesale_price.sum()

In [ ]:
df_month = df_month.to_frame()

In [ ]:
df_month=df_month.reset_index()

In [ ]:
df_month.info()

In [ ]:
df_june = df_month.drop([6])

In [ ]:
df_june

In [ ]:
#sns.set(style='darkgrid')
#ax = sns.barplot(x="created_date", y='shipper_wholesale_price', data= df_month, color='blue')


fig, ax = plt.subplots(figsize = (12,6))    
fig = sns.barplot(x = "created_date", y = "shipper_wholesale_price", data = df_june, color= 'salmon', ax=ax)

x_dates = df_month['created_date'].dt.strftime('%m-%d-%Y').sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

ylabels = ['{:,.2f}'.format(y) for y in ax.get_yticks()]
ax.set_yticklabels(ylabels)


plt.tight_layout()


In [ ]:
import pyodbc
from pathlib import Path
from sqlalchemy import create_engine

In [ ]:
cnxn = pyodbc.connect(server = 'bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      database = 'ar_reports',
                      UID = 'admin',
                      PWD = 'N19lrqxnurTUJLJT6GFe')

In [ ]:
sql_ar = pd.read_sql("SELECT * FROM ar_data", cnxn)

In [ ]:
sql_ar.to_excel('../ar_reports_manual_sql_export/ar_all.xlsx')

In [ ]:
ar_headwaters = sql_ar[sql_ar['client'] == 'Headwaters']

In [ ]:
ar_headwaters.groupby('date').sum()

In [ ]:
sql_df = sql_ar[sql_ar['license_number'] == 'C11-0000314-LIC']

In [ ]:
sql_ar[sql_ar['license_number'] == 'CDPH-10003525']

In [ ]:
sql_df.groupby(by='client').sum().style.format('{:,}')

In [ ]:
# sql_df.groupby(by='date').sum()

In [ ]:
# sql_df.groupby(by='client').count()

In [ ]:
# sql_df[sql_df['client'] == 'Headwaters'].groupby('date').sum().style.format('{:,}')

In [ ]:
# sql_df[sql_df['client'] == 'Green Rocket'].groupby('date').sum().style.format('{:,}')

In [ ]:
s_curr = sql_df.resample('M', on='date').current.sum()
s_30 = sql_df.resample('M', on='date')['30_days'].sum()
s_60 = sql_df.resample('M', on='date')['60_days'].sum()
s_90 = sql_df.resample('M', on='date')['90_days'].sum()
s_ovr = sql_df.resample('M', on='date').over_90_days.sum()

In [ ]:
s_all = pd.concat([s_curr, s_30, s_60, s_90, s_ovr], axis=1)

In [ ]:
s_bal = sql_df.resample('M', on='date')['bal_out'].sum()

In [ ]:
s_all = pd.concat([s_all, s_bal], axis=1)

In [ ]:
s_all = s_all.reset_index()

In [ ]:
ax = s_all.reset_index().plot(x = "date", y = ['current', '30_days', '60_days', '90_days','over_90_days'], kind="bar",stacked=True,figsize=(10,10))

x_labels = s_all['date'].dt.strftime('%Y-%m-%d')
ax.set_xticklabels(x_labels)

plt.show()

In [ ]:


# ax = df.plot(kind='bar')
# x_labels = df.index.strftime('%b')
# ax.set_xticklabels(x_labels)

# x_dates = s_all['date'].dt.strftime('%Y-%m-%d').sort_values().unique()
# ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

# fig, ax = plt.subplots(figsize=(10, 10))

# fig1 = sns.barplot(x="date", y='over_90_days', data=s_all, color='darkgreen')

# fig2 = sns.barplot(x="date", y='90_days', data=s_all, color='lightgreen')

# fig3 = sns.barplot(x="date", y='60_days', data=s_all, color='darkblue')

# fig4 = sns.barplot(x="date", y='30_days', data=s_all, color='lightblue')

# fig5 = sns.barplot(x="date", y='current', data=s_all, color='purple')

# fig3 = sns.barplot(x="date", y='90_days', data=s_all, color='lightgreen')

# fig4 = sns.barplot(x="date", y='over_90_days', data=s_all, color='darkgreen')

# x_dates = s_all['date'].dt.strftime('%Y-%m-%d').sort_values().unique()
# ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

# ylabels = ['{:,.2f}'.format(y) for y in ax.get_yticks()]
# ax.set_yticklabels(ylabels)

# plt.tight_layout()

# df=pd.DataFrame(earnings,index=employees)

# df.plot(kind="bar",stacked=True,figsize=(10,8))
# plt.legend(loc="lower left",bbox_to_anchor=(0.8,1.0))
# plt.show()